<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at 'In [7]'.</span>

In [1]:
from typing import *
from yspecies import *
from yspecies.enums import *
from yspecies.dataset import *
from yspecies.misc import *

In [2]:
from dataclasses import dataclass
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
#settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import pprint
pp = pprint.PrettyPrinter(indent=4)

### Parameters ###
Will be overriden by comparisons.yaml

In [4]:
#Parameters
threshold = 0.3


In [5]:
# Parameters
threshold = 0.3


### Load pathes ###

In [6]:
from pathlib import Path
locations: Locations = Locations("./") if Path("./data").exists() else Locations("../")

In [7]:
external = shap_results = locations.output / "external" 
shap = external / "shap"
linear = external / "linear"
causality = external / "causality"

TypeError: unsupported operand type(s) for /: 'Output' and 'str'

### Load linear ###

In [ ]:
def lag_linear(df: pd.DataFrame, sign: int)->pd.DataFrame:
    selected = df[df["maxlifespan_sign"]==sign][["symbol","organ","maxlifespan_adjpval","maxlifespan_r2_adj"]].sort_values(by="maxlifespan_r2_adj", ascending=False).drop_duplicates()
    selected["organ"] = selected.apply(lambda row: row["organ"] + " (" + str(round(row["maxlifespan_r2_adj"],4)) +")", axis=1)
    return selected[["symbol","organ"]].groupby("symbol", as_index=False).agg({'organ': ', '.join})


def pro_linear(df: pd.DataFrame)->pd.DataFrame:
    return lag_linear(df, 1)
   
def anti_linear(df: pd.DataFrame)->pd.DataFrame:
    return lag_linear(df, -1)


In [ ]:
linear_organ = load_table(linear / 'with_organ_indicator_columns.tsv').set_index("reference_gene")
anti_linear_organ = anti_linear(linear_organ)
pro_linear_organ = pro_linear(linear_organ)
pro_linear_organ.head(10)


In [ ]:
#linear_blk = load_table(linear /'Significant in B,L,K.tsv').set_index("reference_gene")
#linear_models = load_table(linear / 'linear_models_on_species_vars.tsv').set_index("reference_gene")
#linear_all = load_table(linear / 'Significant in all organs.tsv').set_index("reference_gene")
#linear_pro = linear_all[linear_all["maxlifespan_sign"]==1].sort_values(by="maxlifespan_r2_adj", ascending = False)
#linear_anti = linear_all[linear_all["maxlifespan_sign"]==-1].sort_values(by="maxlifespan_r2_adj", ascending = True)

### Load shap ###

In [ ]:
def pro_genes(df: pd.DataFrame, threshold:float = 0.3) -> pd.DataFrame:
    return df[df["kendall_tau_to_max_lifespan"] >= threshold].sort_values(by="kendall_tau_to_max_lifespan", ascending=False)

def lifehistory(row, extra: list, threshold: float):
    def get(col: str):
        return col +" (" + str(round(row[col], 4))+")" if abs(row[col])>=abs(threshold) else ""
    return ', '.join([get(e).replace("kendall_tau_to_", "") for e in extra if get(e) != ""])

def pro(df: pd.DataFrame, threshold:float = 0.3) -> pd.DataFrame:
    main = ['ids','name', 'gain_score_to_max_lifespan', 'kendall_tau_to_max_lifespan']
    extra = ["kendall_tau_to_gestation_days", "kendall_tau_to_mass_g", "kendall_tau_to_temperature_celsius", "kendall_tau_to_metabolic_rate"]
    genes =  pro_genes(df, threshold)[main]    
    genes["life_history"] = df.apply(lambda row: lifehistory(row, extra, threshold), axis=1)
    return genes

def anti_genes(df: pd.DataFrame, threshold:float = -0.3) -> pd.DataFrame:
    return df[df["kendall_tau_to_max_lifespan"]<=threshold].sort_values(by="kendall_tau_to_max_lifespan", ascending=True)

def anti(df: pd.DataFrame, threshold:float = -0.3) -> pd.DataFrame:
    main = ['ids','name', 'gain_score_to_max_lifespan', 'kendall_tau_to_max_lifespan']
    extra = ["kendall_tau_to_gestation_days", "kendall_tau_to_mass_g", "kendall_tau_to_temperature_celsius", "kendall_tau_to_metabolic_rate"]   
    genes = anti_genes(df, threshold)[main]    
    genes["life_history_kendal_tau"] = df.apply(lambda row: lifehistory(row, extra, threshold) , axis=1)
    return genes

In [ ]:
anton_5_5 = pd.read_csv(shap / "5_tissues_anton_species_5_bootstraps.csv", index_col=0)
eugen_5_5 = pd.read_csv(shap / "5_tissues_data_11_06_eugene_species.csv", index_col=0)
anton_7_4 = pd.read_csv(shap / "7_tissues_anton_species_4_bootstraps.csv", index_col=0)
anton_7_5 = pd.read_csv(shap / "7_tissues_anton_species_5_bootstraps.csv", index_col=0)
current = anton_5_5
pro(current, threshold)

In [ ]:
anti(current, -threshold)

### Causality ###

### Intesection ###

In [ ]:
causal_anton_5_5 = pd.read_csv(causality / "causal_relations_5_tissues_species_5.csv")
fathers = (causal_anton_5_5[causal_anton_5_5["predicate"] == "is father of"])[["node1"]].drop_duplicates().rename(columns={"node1": "name"})
fathers

### Causal merging of pro and anti longevity genes ###

In [ ]:
pro_causal_merge = pro(current).merge(fathers, on="name", how="left", indicator = True).sort_values(by="_merge", ascending=False)
pro_causal_merge["_merge"] = pro_causal_merge["_merge"]=="both"
pro_causal = pro_causal_merge \
  .rename(columns={"_merge": "causal", "ids": "reference_gene", "name": "symbol"}) \
  .set_index("reference_gene") \
  .sort_values(by="kendall_tau_to_max_lifespan", ascending=False)
pro_causal

In [ ]:
anti_causal_merge = anti(current).merge(fathers, on="name", how="left", indicator = True).sort_values(by="_merge", ascending=False)
anti_causal_merge["_merge"] = anti_causal_merge["_merge"]=="both"
anti_causal = anti_causal_merge \
  .rename(columns={"_merge": "causal", "ids": "reference_gene", "name": "symbol"}) \
  .set_index("reference_gene") \
  .sort_values(by="kendall_tau_to_max_lifespan", ascending=False)
anti_causal

### Join with linear models ###

In [ ]:
def shap_with_linear(shap_causal: pd.DataFrame, linear_organ: pd.DataFrame):
  return shap_causal.reset_index().merge(linear_organ,on="symbol", how="left") \
    .rename(columns={"organ": "organs (r^2) in linear models", "life_history": "life_history_kendal_tau", "kendall_tau_to_max_lifespan": "MLS_kendall_tau", "gain_score_to_max_lifespan": "MLS_gain_score"}) \
    .sort_values(by="MLS_kendall_tau", ascending = False) \
    .set_index("reference_gene")

In [ ]:
pro_all =  shap_with_linear(pro_causal, pro_linear_organ)
pro_all

In [ ]:
anti_all =  shap_with_linear(anti_causal, anti_linear_organ).sort_values("MLS_kendall_tau")
anti_all

In [ ]:
pro(anton_7_4)

In [ ]:
intersections = locations.output / "intersections"
pro_all.to_csv(intersections / "pro_intersections.tsv", sep = "\t", index = True)
anti_all.to_csv(intersections / "anti_intersections.tsv", sep = "\t", index = True)

In [ ]:
pro_all_ext = pro_all.merge(pro(anton_7_4).rename(columns={"ids": "reference_gene", "name": "symbol"}), \
  on=["reference_gene", "symbol"], how="outer", indicator = True) \
  .sort_values(by="_merge", ascending=False)
pro_all_ext.to_csv(intersections / "pro_7_4_intersections.tsv", sep = "\t", index = False)
pro_all_ext

In [ ]:
anti_all_ext_7_5 = anti_all.merge(anti(anton_7_5).rename(columns={"ids": "reference_gene", "name": "symbol"}), \
  on=["reference_gene", "symbol"], how="outer", indicator = True) \
  .sort_values(by="_merge", ascending=False)
anti_all_ext_7_5.to_csv(intersections / "anti_7_5_intersections.tsv", sep = "\t", index = False)
anti_all_ext_7_5

## Genage annotations ##

In [ ]:
genage_dir = locations.input.input / "annotations" / "genage"

genage_conversions = pd.read_csv(genage_dir  / "genage_conversion.tsv", sep="\t")
genage = genage_conversions[["Ensembl","Gene Symbol", "Organism", "Lifespan Effect", "Longevity Influence", "Method"]].drop_duplicates()
genage.head(10)

In [ ]:
genage["Ensembl"].drop_duplicates().shape

In [ ]:
genage_genes = pd.read_csv(genage_dir  / "genage_orthologs" / "all.tsv", sep="\t").rename(columns={"Homo_sapiens":"reference_gene"})
genage_genes.head(10)

In [ ]:
genage_humanized = genage.head(0)
for c in genage_genes.columns[1:]:
    col = c.replace("_", " ")
    selected_genes = genage_genes[["reference_gene",c]].rename(columns = {c:"Ensembl"}).dropna()
    selected_genes["Ensembl"] = selected_genes.apply(lambda row: row["Ensembl"].split(";"),1)
    genage_org = genage[genage["Organism"]==col]
    merged = selected_genes.explode("Ensembl").merge(genage_org, on="Ensembl", how="inner")
    genage_humanized = pd.concat([genage_humanized, merged])
genage_humanized = genage_humanized.drop_duplicates()
genage_humanized.head(10)

In [ ]:
pro_genage = pro_all_ext_7_5.merge(genage_humanized, on="reference_gene")
pro_genage

In [ ]:
anti_genage = anti_all_ext_7_5.merge(genage_humanized, on="reference_gene")
anti_genage

In [ ]:
genage_human = pd.read_csv(genage_dir / "genage_human.csv")
genage_human.shape

In [ ]:
pro_human_genage = genage_human.merge(pro_all_ext_7_5, on="symbol")
pro_human_genage

In [ ]:
anti_human_genage = genage_human.merge(anti_all_ext_7_5, on="symbol")
anti_human_genage

In [ ]:
pro_genage.to_csv(intersections / "genage_pro_model_intersections.tsv", sep = "\t", index = False)
anti_genage.to_csv(intersections / "genage_anti_model_intersections.tsv", sep = "\t", index = False)
pro_human_genage.to_csv(intersections / "genage_pro_human_intersections.tsv", sep = "\t", index = False)
anti_human_genage.to_csv(intersections / "genage_anti_human_intersections.tsv", sep = "\t", index = False)